Prepare dependencies and global variables

In [30]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, concatenate
from keras.models import Model

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

TSV_DIR = "/content/drive/My Drive/Text Classification/Testground/data/imdb/labeledTrainData.tsv"
GLOVE_DIR = "/content/drive/My Drive/Text Classification/Testground/data/glove/"

Mounted at /content/drive


Prepare dependent functions

In [0]:
def get_train_data(dir):
  data_train = pd.read_csv(dir, sep='\t')
  return data_train

def get_pretrained_glove_vector(dir):
  embeddings_index = {}
  f = open(os.path.join(dir, 'glove.6B.100d.txt'))

  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))
  return embeddings_index

# remove some unwanted characters
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(b'\\\\', b'', string)   
    string = re.sub(b"\'", b"", string)    
    string = re.sub(b"\"", b"", string)    
    return string.strip().lower()

def standardized_text(data_train):
  # parser train data using BeautifulSub and remove html tags
  text = BeautifulSoup(data_train.review[idx])
  unclean_str = text.get_text().encode('ascii','ignore')
  cleaned_str = clean_str(unclean_str)
  # decode cleaned string to bytes-like type
  return cleaned_str.decode('utf-8')

def create_simplified_cnn_network(filter, activate_function="relu"):
  l_cov1= Conv1D(128, 5, activation='relu')(filter)
  l_pool1 = MaxPooling1D(5)(l_cov1)
  l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
  l_pool2 = MaxPooling1D(5)(l_cov2)
  l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
  l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
  l_flat = Flatten()(l_pool3)
  l_dense = Dense(128, activation='relu')(l_flat)
  preds = Dense(2, activation='softmax')(l_dense)

  model = Model(sequence_input, preds)
  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['acc'])
  return model

def create_deeper_cnn_network(filter, activate_function="relu"):
  l_concatenate = concatenate(filter, axis=1)
  l_cov1= Conv1D(128, 5, activation='relu')(l_concatenate)
  l_pool1 = MaxPooling1D(5)(l_cov1)
  l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
  l_pool2 = MaxPooling1D(30)(l_cov2)
  l_flat = Flatten()(l_pool2)
  l_dense = Dense(128, activation='relu')(l_flat)
  preds = Dense(2, activation='softmax')(l_dense)

  model = Model(sequence_input, preds)
  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['acc'])
  return model

Prepare data to train

In [22]:
data_train = get_train_data(TSV_DIR)
texts = []
labels = []

for idx in range(data_train.review.shape[0]):
  texts.append(standardized_text(data_train))
  labels.append(data_train.sentiment[idx])
    

# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# split the data into a training set and a validation set
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# The training data & label for each data set
# training_data  = x_train
# training_label = y_train
training_data = data[:-nb_validation_samples]
training_label = labels[:-nb_validation_samples]

# The validation data & label
# validation_data     = x_val
# validation_label    = y_val
validation_data = data[-nb_validation_samples:]
validation_label = labels[-nb_validation_samples:]


# prepare embedding matrix
embeddings_index = get_pretrained_glove_vector(GLOVE_DIR)
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector        

# load pre-trained word embeddings into an Embedding layer
# By setting trainable = True, the network would learn the embedding by itself.     
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)


sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

print("...Data is ready to train")

Found 80566 unique tokens.
Shape of data tensor: (25000, 1000)
Shape of label tensor: (25000, 2)
Total 400000 word vectors in Glove 6B 100d.
...Data is ready to train


Train data using a simplified convolutional approach

In [0]:
# applying a simplified convolutional approach
activate_function = "relu"
model = create_simplified_cnn_network(embedded_sequences, activate_function)
print("Model fitting - simplified convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=128)

Train data using Deeper Convolutional neural network

In [0]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

for fsz in filter_sizes:
    l_conv = Conv1D(filters=128,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

activate_function = "relu"
model = create_deeper_cnn_network(convs, activate_function)
print("Model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=50)

Model fitting - more complex convolutional neural network
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1000, 100)    8056700     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_30 (Conv1D)              (None, 998, 128)     38528       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 997, 128)     51328       embedding_2[0][0]                
__________________________________